<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/NN-online/blob/main/practice0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practice 1

In this  exercise we __modify__ the feedforward neural network we used  to recognize handwrittne digits (the MNIST data).  The modification involves the activation function for the intermediate layers. Instead of hardcoding them as we did before we supply them as parameters. 

For example if we have 3 layers with sigmoid for the intermediate and softmax for the last layer we supply the activations as a list of pair of functions. Each pair refers to the activation and its derivative.
 
```
activations=[(sigmoid,dsigmoid),(sigmoid,dsigmoid),(softmax,None)]

```
There are two functions that need change: forward_propagation and backward_propagation. Also, the calls to these functions should change because they take on extra parameter

### Packages

Since we are building the network from scratch (i.e. without using any framework) we take advantaged of the GPU on collab by using __cupy__ instead of __numpy__

In [1]:
using_cupy=0
import numpy as np
import random
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer



In [ ]:
from google.colab import files
file=files.upload()
!mkdir /root/.kaggle
!mv kaggle.json  /root/.kaggle
!kaggle datasets download -d uciml/iris
!unzip iris.zip

In [2]:
df=pd.read_csv("Iris.csv")

In [3]:
df=df.drop('Id',axis=1)
df = shuffle(df)


In [4]:
species_lb = LabelBinarizer()
labels = species_lb.fit_transform(df.Species.values)

In [5]:
features = df[df.columns[0:4]].values

In [6]:
x_train=features[0:120]
x_test=features[120:150]
y_train=labels[0:120]
y_test=labels[120:150]

### Activation Functions

Typically in such a setting the activation function for the inner layers could be sigmoid, relu,... and the activation for the last layer would be softmax which is a generalization of the sigmoid for multiclass inputs. Let $C$ be the set of classes and $c_1\in C$ a given class the probability of $c_1$ becomes (where $z_i$ is the logit corrsponding to class $i$)
\begin{align*}
softmax=\frac{e^{z_1}}{\sum_i e^{z_i}}
\end{align*}

In [7]:
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s
def softmax(x):
    m=np.max(x)
    return np.exp(x-m)/np.sum(np.exp(x-m),axis=-1).reshape(x.shape[0],1)
    #return np.array(tf.nn.softmax(np.asnumpy(x)).numpy())


## Derivative of the sigmoid

The derivative of the sigmoid with respect to its parameter is easily obtained
\begin{align*}
\frac{\partial\sigma(z)}{\partial z}&=\frac{1}{\partial z}\frac{1}{1+e^{-z}}\\
                                    &=\frac{e^{-z}}{(1+e^{-z})^2}\\
                                    &=\frac{1}{1+e^{-z}}\cdot \frac{e^{-z}}{1+e^{-z}}\\
                                    &=\sigma(z)(1-\sigma(z))
\end{align*}


In [8]:
#derivative of the sigmoid
def dsigmoid(x):
  return sigmoid(x)*(1-sigmoid(x))

The usual cross Entropy cost. Unlike the evaluate function both the "true" labels and the output of our model are in one-hot encoding.
__NOTE__: the call to forward_propagation was changed to take an extra parameter: activations

In [17]:
def compute_cost(X,Y,b,w,activations):
    
    m = Y.shape[0] # number of example

    # Compute the cross-entropy cost
    As,_=forward_propagation(X,b,w,activations)
    # recall that As contains the "output" of all layers
    # including the input As[0] and the final output As[-1]
    output=As[-1]
    logprobs = np.log(output)*Y
    cost = -np.sum(logprobs)/m

    count=0
    for i in range(output.shape[0]):
        if (np.argmax(output[i,:])==np.argmax(Y[i,:])):
            count=count+1

    
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect. 
                                # E.g., turns [[17]] into 17 
    
    return cost,count


The weights are initialized randomly and the biases are initially set to zero

In [11]:
def initialize_parameters(width):    
    weights=[]
    biases=[]
    for i in range(len(width)-1):
        w=np.random.randn(width[i],width[i+1])
        b=np.zeros((width[i+1],))
        weights.append(w)
        biases.append(b)

    return biases,weights


### __NEEDS CHANGE__: forward_propagation
Forward propagation over all the layers but also retain the intermediate results.
1. takes an extra parameter: list of activation functions
1. use the proper activation for each layer
1. returns logits Z in addition to A

In [12]:
def forward_propagation(X,biases,weights,activations):
    a=X
    As=[X]
    Z=[X]
    para=zip(weights,biases,activations)
    for i,(w,b,act) in enumerate(para):
       z=np.dot(a,w)+b
       ##apply the activation, act[0]. act[1] is the derivative
       a=act[0](z)
       #if i==len(biases)-1:
         
       #  a=softmax(z)
       #else:
       #  a=sigmoid(z)
       As.append(a)
       Z.append(z)
    return As,Z

## Backpropagation

Note how the gradient of a given layer depends on the gradient of the __next__ layer. This means that when computing the gradients we start from the __last__ layer and move __backwards__.
\begin{align*}
\Delta^l_{mn}&=\sum_k \Delta^{l+1}_{mk}W^l_{nk}\theta^l_{mn}\\
dW_{ij}&=\frac{1}{m}\sum_m\Delta^{l+1}_{mj}A^l_{mi}\\
dB_{j}&=\frac{1}{m}\sum_m\Delta^{l+1}_{mj}
\end{align*}

Or in matrix notation
\begin{align*}
\Delta^l&=\Delta^{l+1}\cdot (W^l)^T\theta^l\\
dW^l&=\frac{1}{m}(A^l)^T\cdot \Delta^{l+1}
\end{align*}

### __NEEDS CHANGE__: Backpropagation

theta depends on the activation of the layer 

In [13]:
def backward_propagation(X,Y,biases,weights,activations):
    
    As,Z=forward_propagation(X,biases,weights,activations)
    m = X.shape[0]
    
    nlayers=len(biases)
    # Loss of the last layer
    dz=As[nlayers]-Y

    gradb=[]
    gradw=[]
    #iterator backwards from the last layer
    for i in range(nlayers-1,-1,-1):
        db=np.sum(dz,axis=0,keepdims=True)/m
        dw=np.dot(As[i].T,dz)/m
        # prepare the computation for the
        # NEXT iteration. If i=0 there is no
        # next iteration.
        if i!=0:
          theta=activations[i-1][1](Z[i])
          dz=np.dot(dz,weights[i].T)*theta
        gradb.insert(0,db)
        gradw.insert(0,dw)
    
    return gradb,gradw

In [14]:
def update_parameters(biases,weights, gradb,gradw, learning_rate):


    for i in range(len(biases)):
        weights[i]=weights[i]-learning_rate*gradw[i]
        biases[i]=biases[i]-learning_rate*gradb[i]

    return biases,weights

### Stochastic Gradient Descent

We deal with the input data in __random batches__. We select a set of random indices and take a slice from X and Y

In [26]:
def GD(X, Y, test_data,width,activations,batch_size,num_iterations, learning_rate,print_cost=False):

    biases,weights=initialize_parameters(width)
    
    for i in range(0, num_iterations):
        cost,count = compute_cost(X,Y,biases,weights,activations)
        for k in range(0,X.shape[0],batch_size):
            ## We CANNOT use np.random.shuffle because
            ## it would randomize x and y DIFFERENTLY
            ## instead get a set of random  indices
            ## and use the SAME indices for x and y
            idx=[random.randint(0,Y.shape[0]-1) for s in range(batch_size)]
            yb=Y[idx,:]
            xb=X[idx,:]
            gradb,gradw=backward_propagation(xb,yb,biases,weights,activations)
            biases,weights=update_parameters(biases,weights,gradb,gradw,learning_rate)    

        if i%20 ==0 : 
            #count_test=evaluate(test_data,test_labels,biases,weights)
            count=count/Y.shape[0]
            if using_cupy:
                print ("Epoch {}: cost={:.2f},train accuracy={:.2f}".
                       format(i,np.asnumpy(cost),np.asnumpy(count)))
            else:
                print ("Epoch {}: cost={:.2f},train accuracy={:.2f}".format(i,cost,count))
                
    return biases,weights

#### Run SGD

In [28]:
n_x=x_train.shape[1]
n_y=y_train.shape[1]
width=[n_x,16,8,n_y]
activations=[(sigmoid,dsigmoid),(sigmoid,dsigmoid),(softmax,None)]

biases,weights= GD(x_train,y_train,x_test,width,activations,batch_size=8,learning_rate=0.01,
                num_iterations =100, print_cost=True)

Epoch 0: cost=1.46,train accuracy=0.34
Epoch 20: cost=0.59,train accuracy=0.88
Epoch 40: cost=0.49,train accuracy=0.93
Epoch 60: cost=0.42,train accuracy=0.96
Epoch 80: cost=0.36,train accuracy=0.97


Returns the number of correct predictions. Note that the output of our model is in one-hot encoding so it has 10 columns and N rows where N is the number of data points whereas test_labels is NOT in one-hot encoding so it has a single column and N rows. For a given row i, argmax returns the column index which has the largest value, i.e. the largest likelyhood


In [30]:
def evaluate(test_data,test_labels,biases,weights,activations):
    As,_=forward_propagation(test_data,biases,weights,activations)
    output=As[-1]
    count=0
    #the output is in one-hot encoding so it has 10 rows
    # and number of data columns where as test_tables 
    # is NOT in one-hot encoding so it has a single row
    for i in range(output.shape[0]):
        if np.argmax(output[i,:])==np.argmax(test_labels[i,:]):
            count=count+1
    return count/test_labels.shape[0]

In [31]:
evaluate(x_test,y_test,biases,weights,activations)

0.9333333333333333

### Missclassified inputs

This function plots some of the misclassified data in order for us to have an idea what when wrong.

In [ ]:
def print_misclassified(test_data,test_labels,biases,weights,activations):
    As,_=forward_propagation(test_data,biases,weights,activations)
    output=As[-1]
    count=0
    fig=plt.figure()
    fig.tight_layout()
    plt.subplots_adjust( wspace=1, hspace=1)

    for i in range(output.shape[0]):
        label=np.argmax(output[i,:])
        if label != test_labels[i,0]:
            if count>40:
                break
            subfig=count%40+1
            count=count+1
            img=np.asnumpy(test_data[i,:]).reshape(28,28)
            t=fig.add_subplot(4,10,subfig)
            #t.set_title(str(i))
            t.set_title(str(label))
            t.axes.get_xaxis().set_visible(False)
            t.axes.get_yaxis().set_visible(False)
            plt.imshow(img,cmap='gray_r')



Prints some of the misclassified digits. The top on each digit shows the (wrong) prediction.

In [ ]:
print_misclassified(test_data,test_labels,biases,weights,activations)
